In [1]:
import pandas as pd
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings


pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [3]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [4]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
/02_regular_reports
beeline_video


In [5]:
data_link = download_response.content

In [6]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [7]:
sheet_names.sheet_names

['15s', '6s']

In [8]:
report_type = 'test'

In [9]:
cols_range = 'A:J' # задаем диапазон полей, которые нам нужны
    # создаем список с названиями полей

In [10]:
df = pd.read_excel(BytesIO(data_link), sheet_name='15s', usecols=cols_range, header=None, skiprows=8)

In [13]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date,Impressions,Clicks,Reach,First Quartile,Midpoint,Third Quartile,Complete views,NaN,NaN
2,2024-08-26 00:00:00,4250,32,1862,3871,3704,3141,2986,NaN,NaN


In [15]:
df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить
        
start_index_list = list(df[df[0]=='Date'].index)  # берем индекс начала таблицы с данными

In [19]:
start_index = start_index_list[0]
name = '15s'

In [70]:
creative_name = df[0][start_index-1]
key = name
# если такое название есть, то добавляем его к базовому названию креатива
if creative_name:
    key = name + '_' + str(creative_name).lower()

df_tmp = df.iloc[start_index+1:]
df_tmp.columns = df.iloc[start_index].str.lower().str.strip() # забираем название полей из файла
df_tmp = df_tmp.rename(columns={'first quartile': '25', 'midpoint': '50', 'third quartile': '75', 'complete views': '100'}) # привоодим их к стандарту
df_tmp = df_tmp[['date', 'impressions','reach', 'clicks',  '25', '50', '75', '100']] # оставляем только нужные поля
df_tmp = df_tmp.reset_index(drop=True) # сбрасываем индексацию
end_index = list(df_tmp[df_tmp['date']==0].index)[0] # получаем окончание датаФрейма
df_tmp = df_tmp.iloc[:end_index] # обрезаем датаФрейм до нужной строки

In [72]:
end_index

28

In [74]:
df_tmp

1,date,impressions,reach,clicks,25,50,75,100
0,2024-08-26 00:00:00,4250,1862,32,3871,3704,3141,2986
1,2024-08-27 00:00:00,2583,768,23,2376,2254,1881,1817
2,2024-08-28 00:00:00,1900,661,24,1747,1665,1378,1321
3,2024-08-29 00:00:00,453,146,10,421,400,321,314
4,2024-08-30 00:00:00,50303,20787,504,47409,45999,45005,43014
5,2024-08-31 00:00:00,54014,22230,556,50973,48602,47620,46467
6,2024-09-01 00:00:00,38608,15464,369,34749,31894,31025,30381
7,2024-09-02 00:00:00,30696,11030,306,28149,25598,24812,24290
8,2024-09-03 00:00:00,23676,10524,233,20724,18755,18039,17570
9,2024-09-04 00:00:00,23902,9633,242,19757,16757,15869,15403


In [78]:
tmp_video_dict = {}
    
sheet_names = pd.ExcelFile(BytesIO(data_link))
cols_range = 'A:J' # задаем диапазон полей, которые нам нужны

 # проходим через цикл по списку названий листов
for name in sheet_names.sheet_names:
        # передаем название листа для парсинга, диапазон колонок, которые нам нужны и заранее подготовленный списко названий полей
        df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range, header=None, skiprows=8) 
        df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить
        
        start_index_list = list(df[df[0]=='Date'].index)  # берем индекс начала таблицы с данными
    
        # Проходим через цикл по каждой таблице, которая содержится на листе
        for num, i in enumerate(start_index_list):
            # забираем индекс строки начала таблицы
            start_index = start_index_list[num]
            
            creative_name = df[0][start_index-1]
            key = name
            # если такое название есть, то добавляем его к базовому названию креатива
            if creative_name:
                key = name + '_' + str(creative_name).lower()
            
            df_tmp = df.iloc[start_index+1:]
            df_tmp.columns = df.iloc[start_index].str.lower().str.strip() # забираем название полей из файла
            df_tmp = df_tmp.rename(columns={'first quartile': '25', 'midpoint': '50', 'third quartile': '75', 'complete views': '100'}) # привоодим их к стандарту
            df_tmp = df_tmp[['date', 'impressions','reach', 'clicks',  '25', '50', '75', '100']] # оставляем только нужные поля
            df_tmp = df_tmp.reset_index(drop=True) # сбрасываем индексацию
            end_index = list(df_tmp[df_tmp['date']==0].index)[0] # получаем окончание датаФрейма
            df_tmp = df_tmp.iloc[:end_index] # обрезаем датаФрейм до нужной строки
            df_tmp['date'] = pd.to_datetime(df_tmp['date']).dt.date # приводим датуВремя в просто дату
            df_tmp['source'] = 'beeline'
            df_tmp['format_type'] = 'video' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
            df_tmp['product'] = key # добавляем название продукта
            df_tmp['report_type'] = report_type
            # сохраняем датаФрейм во временный словарь 
            # ключ - это название продукта (15s, 6s и тд) + дополнительное название креатива(если оно есть)
    
            tmp_video_dict[key] = df_tmp

In [79]:
tmp_video_dict

{'15s': 1         date impressions  reach clicks     25     50     75    100   source  \
 0   2024-08-26        4250   1862     32   3871   3704   3141   2986  beeline   
 1   2024-08-27        2583    768     23   2376   2254   1881   1817  beeline   
 2   2024-08-28        1900    661     24   1747   1665   1378   1321  beeline   
 3   2024-08-29         453    146     10    421    400    321    314  beeline   
 4   2024-08-30       50303  20787    504  47409  45999  45005  43014  beeline   
 5   2024-08-31       54014  22230    556  50973  48602  47620  46467  beeline   
 6   2024-09-01       38608  15464    369  34749  31894  31025  30381  beeline   
 7   2024-09-02       30696  11030    306  28149  25598  24812  24290  beeline   
 8   2024-09-03       23676  10524    233  20724  18755  18039  17570  beeline   
 9   2024-09-04       23902   9633    242  19757  16757  15869  15403  beeline   
 10  2024-09-05       22320  10287    224  19552  17227  16194  15694  beeline   
 11  2024

In [24]:
# источник Beeline
# тип отчета Видео реклама
# Функция для обработки статистики по дням для обычных размещений 
# Каждый креатив на отдельном листе

def get_beeline_video_report(data_link, report_type):
    tmp_video_dict = {}
        
    sheet_names = pd.ExcelFile(BytesIO(data_link))
    cols_range = 'A:J' # задаем диапазон полей, которые нам нужны
    
     # проходим через цикл по списку названий листов
    for name in sheet_names.sheet_names:
        # передаем название листа для парсинга, диапазон колонок, которые нам нужны и заранее подготовленный списко названий полей
        df = pd.read_excel(BytesIO(data_link), sheet_name=name, usecols=cols_range, header=None, skiprows=8) 
        df = df.fillna(0)  #заполяем пустые строки, чтобы затем их удалить
        
        start_index_list = list(df[df[0]=='Date'].index)  # собираем список индексов, где есть название Date
    
        # Проходим через цикл по каждой таблице, которая содержится на листе
        for num, i in enumerate(start_index_list):
            # забираем индекс строки начала таблицы
            start_index = start_index_list[num] # берем индекс начала таблицы с данными
            creative_name = df[0][start_index-1] # забираем значение из ячейки над полем Дата 
            key = name 
            if creative_name:
                key = name + '_' + str(creative_name).lower().str.strip() # если такое название есть, то добавляем его к базовому названию креатива
                
            df_tmp = df.iloc[start_index+1:]
            df_tmp.columns = df.iloc[start_index].str.lower().str.strip() # забираем название полей из файла
            # привоодим их к единому стандарту
            df_tmp = df_tmp.rename(columns={'first quartile': '25', 'midpoint': '50', 'third quartile': '75', 'complete views': '100'}) 
            df_tmp = df_tmp[['date', 'impressions','reach', 'clicks',  '25', '50', '75', '100']] # оставляем только нужные поля
            df_tmp = df_tmp.reset_index(drop=True) # сбрасываем индексацию
            end_index = list(df_tmp[df_tmp['date']==0].index)[0] # получаем окончание датаФрейма
            df_tmp = df_tmp.iloc[:end_index] # обрезаем датаФрейм до нужной строки
            df_tmp['date'] = pd.to_datetime(df_tmp['date']).dt.date # приводим датуВремя в просто дату
            df_tmp['source'] = 'beeline'
            df_tmp['format_type'] = 'video' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
            df_tmp['product'] = key # добавляем название продукта
            df_tmp['report_type'] = report_type
            # сохраняем датаФрейм во временный словарь 
            # ключ - это название продукта (15s, 6s и тд) + дополнительное название креатива(если оно есть)
    
            tmp_video_dict[key] = df_tmp  
        
    return pd.concat(tmp_video_dict, ignore_index=True)